In [1]:
import numpy as np
import numpy.linalg as lg

In [2]:
def pde_Elliptic(f,g,BFx,Domain,Mx,My):
    """
    The matrix generator of the Elliptic PDE  #Ax=b
 
    u_xx + u_yy + g(x,y)u = f(x,y) over the region
    D = [x0,xf]x[y0,yf] with the boundary conditions:
    u(x0,y) = bx0(y), u(xf,y) = bxf(y)
    u(x,y0) = by0(x), u(x,yf) = byf(x)
    Mx : the number of subintervals along x axis
    My : the number of subintervals along y axis
 
    BFx = [bx0,bxf,by0,byf]  #邊界函數
    Domain = [x0,xf,y0,yf]
     
    The output A, b the matrix and vector of the corresponding linear system
    """
 
    n = (Mx-1)*(My-1) #The number of variables
    A = np.zeros([n,n])
    b = np.zeros([n,1])
     
    bx0, bxf, by0, byf = BFx[0], BFx[1], BFx[2], BFx[3]
    x0, xf, y0, yf = Domain[0], Domain[1], Domain[2], Domain[3]
 
    hx = float(xf-x0)/Mx #delta x
    hy = float(yf-y0)/My #delta y
 
    x = [x0+j*hx for j in range(1,Mx)]
    y = [y0+i*hy for i in range(1,My)]
 
    h2x = 1.0/(hx**2)
    h2y = 1.0/(hy**2)
     
    def ij2k(i,j): #Transfer (i,j) 2D-index to 1D index
        return i*(Mx-1)+j #對應到x矩陣的第幾個位置
    l = -1
    for i in range(My-1):
        for j in range(Mx-1):
            l += 1
            k = ij2k(i,j)
            b[k] += f(x[j],y[i])
            #set the center coefficient
            A[l][k] = g(x[j],y[i])-2*(h2x+h2y)
            #set the up coefficient
            if i==0:
                b[k] -= h2y*by0(x[j])
            else:
                ku = ij2k(i-1,j)
                A[l][ku] += h2y
            #set the low coefficient
            if i==(My-2):
                b[k] -= h2y*byf(x[j])
            else:
                kd = ij2k(i+1,j)
                A[l][kd] += h2y
            #set the left coefficient
            if j==0:
                b[k] -= h2x*bx0(y[i])
            else:
                kl = ij2k(i,j-1)
                A[l][kl] += h2x
            #set the right coefficient
            if j==(Mx-2):
                b[k] -= h2x*bxf(y[i])
            else:
                kr = ij2k(i,j+1)
                A[l][kr] += h2x
    return A, b

In [3]:
BFx = [lambda y:np.exp(y)-np.cos(y), lambda y:np.exp(y)*np.cos(4)-np.exp(4)*np.cos(y),
       lambda x:np.cos(x)-np.exp(x), lambda x: np.exp(4)*np.cos(x) - np.exp(x)*np.cos(4)]
Domain = [0,4,0,4]

In [4]:
A,b = pde_Elliptic(lambda x,y:0.0, lambda x,y:0.0, BFx,Domain,20,20)  #f = 0, g = 0
#矩陣很大

In [5]:
print A.shape, len(b)

(361, 361) 361


In [6]:
A

array([[-100.,   25.,    0., ...,    0.,    0.,    0.],
       [  25., -100.,   25., ...,    0.,    0.,    0.],
       [   0.,   25., -100., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ..., -100.,   25.,    0.],
       [   0.,    0.,    0., ...,   25., -100.,   25.],
       [   0.,    0.,    0., ...,    0.,   25., -100.]])

In [7]:
x = lg.solve(A,b)

In [8]:
X = x.reshape([19,19])
print X.shape

(19, 19)


In [9]:
x

array([[-5.32907052e-17],
       [-3.37226095e-01],
       [-7.78140677e-01],
       [-1.33109263e+00],
       [-2.00569310e+00],
       [-2.81370519e+00],
       [-3.77012781e+00],
       [-4.89449303e+00],
       [-6.21240111e+00],
       [-7.75732475e+00],
       [-9.57272339e+00],
       [-1.17145210e+01],
       [-1.42540159e+01],
       [-1.72813116e+01],
       [-2.09093768e+01],
       [-2.52788749e+01],
       [-3.05639213e+01],
       [-3.69789504e+01],
       [-4.47868352e+01],
       [ 3.37226095e-01],
       [-2.29543455e-16],
       [-4.47460800e-01],
       [-1.01170252e+00],
       [-1.69999505e+00],
       [-2.52124067e+00],
       [-3.48708021e+00],
       [-4.61321126e+00],
       [-5.92093711e+00],
       [-7.43897157e+00],
       [-9.20553323e+00],
       [-1.12707744e+01],
       [-1.36996045e+01],
       [-1.65749843e+01],
       [-2.00017914e+01],
       [-2.41113765e+01],
       [-2.90669618e+01],
       [-3.50700520e+01],
       [-4.23680543e+01],
       [ 7.7

In [10]:
A,b = pde_Elliptic(lambda x,y:0.0, lambda x,y:0.0, BFx,Domain,100,100)

In [11]:
print A.shape

(9801, 9801)


In [12]:
x = lg.solve(A,b)

In [13]:
X = x.reshape([99,99])
print X.shape

(99, 99)


In [14]:
def pde_poisson(f,g,BFx,Domain,Mx,My,tol=10**(-8),MaxIter=100):
    """
    The PDE solver of u_xx + u_yy + g(x,y)u = f(x,y) over the region
    D = [x0,xf]x[y0,yf] with the boundary conditions:
    u(x0,y) = bx0(y), u(xf,y) = bxf(y)
    u(x,y0) = by0(x), u(x,yf) = byf(x)
    Mx : the number of subintervals along x axis
    My : the number of subintervals along y axis
 
    tol : the tolerance
    MaxIter : the Maximal number of the iteration
     
    The output
    u : u(x_j,y_i)
    x : the uniform grids of x-axis
    y : the uniform grids of y-axis 
    """
 
    bx0, bxf, by0, byf = BFx[0], BFx[1], BFx[2], BFx[3]
    x0, xf, y0, yf = Domain[0], Domain[1], Domain[2], Domain[3]
     
    hx = float(xf-x0)/Mx
    hy = float(yf-y0)/My
 
    x = [x0+j*hx for j in range(1,Mx)]
    y = [y0+i*hy for i in range(1,My)]
 
    Mx, My = Mx+1, My+1
 
    u = np.zeros([My,Mx])
    F = np.zeros([My,Mx])
    G = np.zeros([My,Mx])
    u0 = np.zeros([My,Mx])
 
    #set boundary condition
    j = 1
    for xj in x:
        u[0,j], u[My-1,j] = by0(xj), byf(xj)
        j+=1
    i = 1
    for yi in y:
        u[i,0], u[i,Mx-1] = bx0(yi), bxf(yi)
        i+=1
 
    #initialize as the average of boundary values
    sum_of_bv = sum(u[0,:])+sum(u[My-1,:])+sum(u[1:My-1,0])+sum(u[1:My-1,Mx-1])
 
    u[1:My-1,1:Mx-1] = float(sum_of_bv)/(2*(Mx+My-2))
 
    #set the f(xj,yi) & g(xj,yi)
    for i in range(1,My-1):
        for j in range(1,Mx-1):
            F[i,j], G[i,j] = f(x[j-1],y[i-1]), g(x[j-1],y[i-1])
 
    dx2, dy2 = hx**2, hy**2
    dxy2 = 2*(dx2+dy2)
    rx, ry = dx2/dxy2, dy2/dxy2
    rxy = rx*dy2
    for itr in range(MaxIter):
        for i in range(1,My-1):
            for j in range(1,Mx-1):
                u[i,j] = ry*(u[i,j+1]+u[i,j-1])+rx*(u[i+1,j]+u[i-1,j])+rxy*(G[i,j]*u[i,j]-F[i,j])
        Err = abs(u-u0)
         
        if (itr>1) & (Err.max()<tol):
            break
        u0 = u
 
    u = u[1:My,1:Mx]
    return u, x, y

In [15]:
u,x,y = pde_poisson(lambda x,y:0.0, lambda x,y:0.0, BFx,Domain,20,20,10**(-8),50)

In [16]:
u.shape

(20, 20)

In [17]:
def pde_poisson_MG(f,g,BFx,Domain,Mx,My):
    """
    The Matrix generator of the iteration method to solve u_xx + u_yy + g(x,y)u = f(x,y) over the region
    D = [x0,xf]x[y0,yf] with the boundary conditions:
    u(x0,y) = bx0(y), u(xf,y) = bxf(y)
    u(x,y0) = by0(x), u(x,yf) = byf(x)
    Mx : the number of subintervals along x axis
    My : the number of subintervals along y axis
    """
 
    bx0, bxf, by0, byf = BFx[0], BFx[1], BFx[2], BFx[3]
    x0, xf, y0, yf = Domain[0], Domain[1], Domain[2], Domain[3]
     
    hx = float(xf-x0)/Mx
    hy = float(yf-y0)/My
 
    x = [x0+j*hx for j in range(1,Mx)]
    y = [y0+i*hy for i in range(1,My)]
 
    Mx, My = Mx+1, My+1
 
    u = np.zeros([My,Mx])
    F = np.zeros([My,Mx])
    G = np.zeros([My,Mx])
    u0 = np.zeros([My,Mx])
 
    #set boundary condition
    j = 1
    for xj in x:
        u[0,j], u[My-1,j] = by0(xj), byf(xj)
        j+=1
    i = 1
    for yi in y:
        u[i,0], u[i,Mx-1] = bx0(yi), bxf(yi)
        i+=1
 
    #initialize as the average of boundary values
    sum_of_bv = sum(u[0,:])+sum(u[My-1,:])+sum(u[1:My-1,0])+sum(u[1:My-1,Mx-1])
 
    u[1:My-1,1:Mx-1] = float(sum_of_bv)/(2*(Mx+My-2))
 
    #set the f(xj,yi) & g(xj,yi)
    for i in range(1,My-1):
        for j in range(1,Mx-1):
            F[i,j], G[i,j] = f(x[j-1],y[i-1]), g(x[j-1],y[i-1])
 
    dx2, dy2 = hx**2, hy**2
    dxy2 = 2*(dx2+dy2)
    rx, ry = dx2/dxy2, dy2/dxy2
    rxy = rx*dy2
     
    def ij2k(i,j): #Transfer (i,j) 2D-index to 1D index
        return (i-1)*(Mx-2)+(j-1)
 
    n = (Mx-2)*(My-2) #The number of variables
    A = np.zeros([n,n])
 
    l=-1
    for i in range(1,My-2):
        for j in range(1,Mx-2):
            l += 1
            k = ij2k(i,j)
            #set the center coefficient
            A[l][k] = rxy*G[i,j]
            #set the up coefficient
            if i>0:
                ku = ij2k(i-1,j)
                A[l][ku] += rx
            #set the low coefficient
            if i<(My--1):
                kd = ij2k(i+1,j)
                A[l][kd] += rx
            #set the left coefficient
            if j>0:
                kl = ij2k(i,j-1)
                A[l][kl] += ry
            #set the right coefficient
            if j<(Mx-1):
                kr = ij2k(i,j+1)
                A[l][kr] += ry
    return A

In [18]:
A = pde_poisson_MG(lambda x,y:0.0, lambda x,y:0.0, BFx,Domain,20,20)

In [19]:
D,V = lg.eig(A)
print max(D)

(0.9123180999541357+0j)
